<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/gsr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install QuantLib-Python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 25.3 MB/s eta 0:00:00


In [ ]:
import QuantLib as ql
import unittest
import math

# Helper to create a flat yield term structure (if not available in utilities.py)
def flat_rate_handle(forward_rate, day_counter, today=None):
    if today is None:
        today = ql.Settings.instance().evaluationDate
    return ql.YieldTermStructureHandle(
        ql.FlatForward(today, ql.QuoteHandle(ql.SimpleQuote(forward_rate)), day_counter)
    )

class GsrTests(unittest.TestCase):

    def setUp(self):
        self.calendar = ql.TARGET()
        self.today = ql.Date(5, ql.October, 2023) # Example date
        ql.Settings.instance().evaluationDate = self.today

    def test_gsr_process(self):
        print("Testing GSR process...")
        ref_date = ql.Settings.instance().evaluationDate

        tol = 1E-8
        reversion_val = 0.01
        model_vol = 0.01

        yts0 = flat_rate_handle(0.00, ql.Actual365Fixed(), ref_date)

        step_dates0 = [] # ql.DateVector()
        vols0 = [model_vol] # ql.DoubleVector()
        reversions0 = [reversion_val] # ql.DoubleVector()

        step_dates1_list = []
        for i in range(1, 60):
            step_dates1_list.append(ref_date + ql.Period(i * 6, ql.Months))

        step_dates1 = ql.DateVector(step_dates1_list)
        vols1 = ql.DoubleVector([model_vol] * (len(step_dates1_list) + 1))
        reversions1 = ql.DoubleVector([reversion_val] * (len(step_dates1_list) + 1))

        T_param = 10.0
        while T_param <= 30.0:
            model = ql.Gsr(yts0, ql.DateVector(step_dates0), ql.DoubleVector(vols0), ql.DoubleVector(reversions0), T_param)
            gsr_process = model.stateProcess()

            model2 = ql.Gsr(yts0, step_dates1, vols1, reversions1, T_param)
            gsr_process2 = model2.stateProcess()

            hw_process = ql.HullWhiteForwardProcess(yts0, reversion_val, model_vol)
            hw_process.setForwardMeasureTime(T_param)

            t_time = 0.5
            while t_time <= T_param - 0.1:
                w_time = 0.0
                while w_time <= t_time - 0.1:
                    xw_val = -0.1
                    while xw_val <= 0.1:
                        dt = t_time - w_time

                        hw_val_exp = hw_process.expectation(w_time, xw_val, dt)
                        gsr_val_exp = gsr_process.expectation(w_time, xw_val, dt)
                        gsr2_val_exp = gsr_process2.expectation(w_time, xw_val, dt)

                        self.assertAlmostEqual(gsr_val_exp, hw_val_exp, delta=tol,
                                               msg=(f"Expectation E(T={T_param})(x({t_time})|x({w_time})={xw_val}) "
                                                    f"HW ({hw_val_exp}) vs Gsr ({gsr_val_exp})"))
                        self.assertAlmostEqual(gsr2_val_exp, hw_val_exp, delta=tol,
                                               msg=(f"Expectation E(T={T_param})(x({t_time})|x({w_time})={xw_val}) "
                                                    f"HW ({hw_val_exp}) vs Gsr2 ({gsr2_val_exp})"))

                        hw_val_var = hw_process.variance(w_time, xw_val, dt)
                        gsr_val_var = gsr_process.variance(w_time, xw_val, dt)
                        gsr2_val_var = gsr_process2.variance(w_time, xw_val, dt)

                        self.assertAlmostEqual(gsr_val_var, hw_val_var, delta=tol,
                                               msg=(f"Variance V(T={T_param})(x({t_time})|x({w_time})={xw_val}) "
                                                    f"HW ({hw_val_var}) vs Gsr ({gsr_val_var})"))
                        self.assertAlmostEqual(gsr2_val_var, hw_val_var, delta=tol,
                                               msg=(f"Variance V(T={T_param})(x({t_time})|x({w_time})={xw_val}) "
                                                    f"HW ({hw_val_var}) vs Gsr2 ({gsr2_val_var})"))
                        xw_val += 0.01
                    w_time += t_time / 5.0
                t_time += T_param / 20.0
            T_param += 10.0

        # Time dependent reversion and volatility (test cases to be added)
        times_arr = ql.Array([1.0, 2.0])
        vols_arr = ql.Array([0.2, 0.3, 0.4])
        reversions_arr = ql.Array([0.50, 0.80, 1.30])

        # GsrProcess constructor: (times, vols, reversions, T=60.0)
        # vols and reversions have n+1 elements if times has n elements
        p = ql.GsrProcess(times_arr, vols_arr, reversions_arr)
        p.setForwardMeasureTime(10.0)
        # ... add more test cases here (as in C++)
        self.assertIsNotNone(p) # Basic check

    def test_gsr_model(self):
        print("Testing GSR model...")
        ref_date = ql.Settings.instance().evaluationDate

        model_vol = 0.01
        reversion_val = 0.01

        step_dates0_list = []
        vols0 = ql.DoubleVector([model_vol])
        reversions0 = ql.DoubleVector([reversion_val])

        step_dates1_list = []
        for i in range(1, 60):
            step_dates1_list.append(ref_date + ql.Period(i * 6, ql.Months))

        step_dates1 = ql.DateVector(step_dates1_list)
        vols1 = ql.DoubleVector([model_vol] * (len(step_dates1_list) + 1))
        reversions1 = ql.DoubleVector([reversion_val] * (len(step_dates1_list) + 1))

        yts = flat_rate_handle(0.03, ql.Actual365Fixed(), ref_date)

        model = ql.Gsr(yts, ql.DateVector(step_dates0_list), vols0, reversions0, 50.0)
        model2 = ql.Gsr(yts, step_dates1, vols1, reversions1, 50.0)
        hw_model = ql.HullWhite(yts, reversion_val, model_vol)

        tol0 = 1E-8

        # In GSR model, zerobond(T, t, y_t) gives P(t,T) given y_t (normalized state at time t)
        # C++: model->zerobond(t, w, yw) -> P(w,t) given y(w)
        # Python: model.zerobond(maturityTime=t, valueTime=w, y_valueTime=yw)

        w_time = 0.1
        while w_time <= 50.0 - 0.1: # Ensure w_time < t_time and t_time <= 50.0
            t_time = w_time + 0.1
            while t_time <= 50.0:
                xw_val = -0.10
                while xw_val <= 0.10:
                    # Calculate y(w): normalized state variable at time w
                    # y(w) = (x(w) - E[x(w)]) / StdDev[x(w)]
                    # E[x(0)] = 0 by convention for the state process.
                    # StdDev[x(0)] is not well-defined (or process starts at x(0)=0).
                    # If w > 0:
                    if w_time > 1e-6: # Avoid division by zero if w_time is effectively 0
                        exp_xw = model.stateProcess().expectation(0.0, 0.0, w_time)
                        std_dev_xw = model.stateProcess().stdDeviation(0.0, 0.0, w_time)
                        if abs(std_dev_xw) < 1e-10: # Avoid division by very small number
                             yw_val = 0.0 if abs(xw_val - exp_xw) < 1e-10 else (xw_val - exp_xw) / 1e-10
                        else:
                             yw_val = (xw_val - exp_xw) / std_dev_xw
                    else: # if w_time is 0, x(0) is given, y(0) would correspond to x(0)
                          # Assuming x(0) = 0 implies y(0) = 0. If xw_val is x(0), then y(0) can be non-zero.
                          # The C++ test implies xw is x(w). For w=0, xw is x(0).
                          # E[x(0)|x(0)=0] = 0, stdDev[x(0)|x(0)=0] = 0 (or not defined this way for GSR model.zerobond)
                          # The GSR model's zerobond expects y_t, the *normalized* state variable at time t.
                          # If w_time = 0, then xw_val = x(0). GSR uses y(t)=(x(t)-alpha(t))/sigma_p(t).
                          # Here, let's assume if w_time is very small, xw_val is effectively y_w,
                          # or it's x(0) and we should pass x(0) to a different overload if available.
                          # model.zerobond uses y if t > 0. If t=0, it uses x0.
                          # Since w_time is our "valueTime", if w_time = 0, yw_val should be x_0.
                          # But the signature is (maturityTime, valueTime, y_valueTime).
                          # This detail needs care. The C++ code uses same yw calculation always.
                          # If stdDeviation(0,0,0) is 0, then this implies an issue.
                          # HullWhite discountBond expects r_t (instantaneous rate at time t).
                          # r_w = x_w + f(0,w). Here f(0,w) is 0.03
                        yw_val = xw_val # This assumes x(0) = y(0) when w=0 for GSR, or that x(w) is directly used as y(w) in this test structure.
                                       # A more robust approach for w=0:
                                       # if w_time < 1e-6: yw_val = xw_val (interpreted as x_0 for HW and y_0 for GSR's internal alpha mapping)
                                       # For HW, rw is used.
                                       # For GSR, y_w is used. If w=0, y_0 = (x_0 - alpha(0))/sigma_p(0). alpha(0) is often 0.
                                       # GsrProcess stdDev(0,0,w) when w=0 is 0.
                                       # The C++ test passes even when w_time =0. This means that internally this case is handled.
                                       # Let's stick to the C++ logic for yw calculation.
                        if w_time > 1e-9:
                            exp_xw = model.stateProcess().expectation(0.0, 0.0, w_time)
                            std_dev_xw = model.stateProcess().stdDeviation(0.0, 0.0, w_time)
                            yw_val = (xw_val - exp_xw) / std_dev_xw if abs(std_dev_xw) > 1e-9 else 0.0
                        else: # effectively w_time = 0
                            yw_val = xw_val # x(0) is not normalized in the same way for y(0) in GSR context.
                                           # Or, Gsr.zerobond uses x0 if valueTime=0. Let's test if QL handles it.
                                           # After testing, QL Python Gsr.zerobond requires y_valueTime.
                                           # It seems there's no direct overload for x_valueTime.
                                           # So, yw_val must be the normalized y.
                                           # If w_time=0, E(x(0))=0, StdDev(x(0))=0 by process def.
                                           # The C++ code structure implies this case should work.
                                           # For HullWhite, r_w = x_w + f(0,w).
                                           # Instantaneous forward rate f(0,w) from yts is 0.03.
                                           # So, r_w = xw_val + 0.03.
                            pass # yw_val needs to be correctly set for GSR if w_time is 0.
                                 # The C++ works because Gsr::zerobond likely has logic for t1=0.
                                 # For python, if valueTime = 0, third param is x0.
                                 # If valueTime > 0, third param is y(valueTime).

                    gsr_val = model.zerobond(t_time, w_time, yw_val)
                    gsr2_val = model2.zerobond(t_time, w_time, yw_val)

                    # HullWhite::discountBond(initialTime, maturityTime, initialRate_r_initialTime)
                    rw_val = xw_val + yts.forwardRate(w_time, w_time, ql.Continuous).rate() # f(w,w) which is r(w)
                    hw_val = hw_model.discountBond(w_time, t_time, rw_val)

                    self.assertAlmostEqual(gsr_val, hw_val, delta=tol0,
                                           msg=(f"Zerobond P({w_time},{t_time}|x={xw_val}/y={yw_val}) "
                                                f"HW ({hw_val}) vs Gsr ({gsr_val})"))
                    self.assertAlmostEqual(gsr2_val, hw_val, delta=tol0,
                                           msg=(f"Zerobond P({w_time},{t_time}|x={xw_val}/y={yw_val}) "
                                                f"HW ({hw_val}) vs Gsr2 ({gsr2_val})"))
                    xw_val += 0.01
                t_time += 2.5
            w_time += 5.0

        # Test standard, nonstandard and jamshidian engine against existing Hull White Jamshidian engine
        expiry_date = self.calendar.advance(ref_date, ql.Period(5, ql.Years))
        tenor = ql.Period(10, ql.Years)

        # Using Euribor6M as an example index
        ibor_index = ql.Euribor6M(yts)
        # swp_idx = ql.EuriborSwapIsdaFixA(tenor, yts) # This constructor might need feller constraints for GSR if used in calibration.
        # For pricing, a generic swap index is fine. Let's use one based on Euribor6M.
        swp_idx = ql.SwapIndex("EuriborSwapIsdaFixA10Y", tenor,
                               ibor_index.fixingDays(), ibor_index.currency(),
                               ibor_index.fixingCalendar(), ibor_index.dayCounter().empty() # Fixed leg tenor used from ibor_index if not specified
                               , ibor_index.businessDayConvention(),
                               ibor_index, yts) # No feller for pricing

        forward_rate = swp_idx.fixing(expiry_date)

        # C++ underlyingSwap(expiryDate) is not directly on SwapIndex in Python.
        # We use MakeVanillaSwap.
        # underlying = swp_idx.underlyingSwap(expiry_date) # Not available
        # Recreate underlying swap logic as in C++ test for underlyingFixed
        underlying_fixed = ql.MakeVanillaSwap(tenor, ibor_index, forward_rate) \
            .withEffectiveDate(swp_idx.valueDate(expiry_date)) \
            .withFixedLegCalendar(swp_idx.fixingCalendar()) \
            .withFixedLegDayCount(swp_idx.dayCounter()) \
            .withFixedLegTenor(swp_idx.fixedLegTenor()) \
            .withFixedLegConvention(swp_idx.fixedLegConvention()) \
            .withFixedLegTerminationDateConvention(swp_idx.fixedLegConvention()) \
            .withNominal(1.0) # C++ default nominal is 1.0

        exercise = ql.EuropeanExercise(expiry_date)
        std_swaption = ql.Swaption(underlying_fixed, exercise)
        non_std_swaption = ql.NonstandardSwaption(std_swaption)

        # Pricing with Hull-White Jamshidian
        hw_jam_engine = ql.JamshidianSwaptionEngine(hw_model, yts)
        std_swaption.setPricingEngine(hw_jam_engine)
        hw_jam_npv = std_swaption.NPV()

        # Pricing with GSR engines
        gsr_non_std_engine = ql.Gaussian1dNonstandardSwaptionEngine(model, 64, 7.0, True, False)
        gsr_std_engine = ql.Gaussian1dSwaptionEngine(model, 64, 7.0, True, False) # Args: model, integrationPoints, stddevs, includeTodaysCashFlows, usebpsSpread
        gsr_jam_engine = ql.Gaussian1dJamshidianSwaptionEngine(model)

        non_std_swaption.setPricingEngine(gsr_non_std_engine)
        gsr_non_std_npv = non_std_swaption.NPV()

        std_swaption.setPricingEngine(gsr_std_engine)
        gsr_std_npv = std_swaption.NPV()

        std_swaption.setPricingEngine(gsr_jam_engine)
        gsr_jam_npv = std_swaption.NPV()

        tol_npv = 5.0e-5
        self.assertAlmostEqual(gsr_non_std_npv, hw_jam_npv, delta=tol_npv,
                               msg=(f"Jamshidian HW NPV ({hw_jam_npv}) vs "
                                    f"G1dNonstandardSwaptionEngine NPV ({gsr_non_std_npv})"))
        self.assertAlmostEqual(gsr_std_npv, hw_jam_npv, delta=tol_npv,
                               msg=(f"Jamshidian HW NPV ({hw_jam_npv}) vs "
                                    f"G1dSwaptionEngine NPV ({gsr_std_npv})"))
        self.assertAlmostEqual(gsr_jam_npv, hw_jam_npv, delta=tol_npv,
                               msg=(f"Jamshidian HW NPV ({hw_jam_npv}) vs "
                                    f"G1dJamshidianEngine NPV ({gsr_jam_npv})"))

if __name__ == '__main__':
    print("Running Python QuantLib GSR tests...")
    unittest.main(argv=['first-arg-is-ignored'], exit=False)